Прежде всего в HTML-шаблоне нужно изменить метод запроса, который отправляет форма: запросы, которые меняют состояние базы данных (создают, изменяют или удаляют записи БД), должны отправляться методом POST.

Настроим HTML-форму так, чтобы она отправляла POST-запросы; view-функцию тоже придётся изменить: теперь она должна извлекать данные из POST-запроса. Помимо этого, в HTML-форму нужно добавить csrf-токен: без него Django отклонит POST-запрос. 

Внесите изменения в шаблон *templates/birthday/birthday.html*:

```html
<!-- birthday/birthday.html -->
...
<!-- Укажите метод отправки запроса. -->
<form method="post">
  <!-- Добавьте тег csrf_token. Без него POST-запрос будет отклонён. -->
  {% csrf_token %}
  {% bootstrap_form form %}
  {% bootstrap_button button_type="submit" content="Отправить" %}
</form>
<!-- Замените метод запроса на POST. -->
{% with data=request.POST %}
... 
```

Во view-функции `birthday()`, в строке, где создаётся объект формы, нужно изменить метод GET на POST: параметры POST-запроса хранятся в объекте `request.POST`.

```py
...
def birthday(request):
    form = BirthdayForm(request.POST or None)
... 
```

Теперь при GET-запросе view-функция будет просто возвращать форму, а при POST-запросе — валидировать данные, полученные из формы, и возвращать заполненную форму.

> В нашем проекте GET-запрос к странице с формой и POST-запрос, отправленный из формы, обрабатываются одной и той же view-функцией — `birthday()`. Но такой подход — не догма.

> Например, HTML-форма может быть размещена на странице *birthday/*, а POST-запрос из формы можно отправлять к адресу *save_it/* — для этого нужно указать атрибут `action="save_it/"` в теге `<form>`. И теперь генерировать HTML-форму будет одна view-функция, а получать и обрабатывать данные из формы — другая, связанная с адресом *save_it/*. Но в приложении **birthday** проще обойтись одной view-функцией.

***
## Сохранение данных из запроса в базу данных

В классе `ModelForm` есть встроенный метод `save()`, он позволяет сохранить данные из формы в БД. После сохранения метод `save()` возвращает сохранённый объект — это можно использовать для подтверждения, что сохранение данных прошло успешно.

Добавим строчку с вызовом метода `save()`:

```py
...
def birthday(request):
    form = BirthdayForm(request.POST or None)
    context = {'form': form}
    if form.is_valid():
        form.save()
        ... 
```

Готово!

Не каждый пользователь может заглянуть в БД; подтверждение того, что объект добавлен в базу, лучше выводить прямо на веб-страницу. Например, можно отображать на странице `id` новой записи: это поле заполняется автоматически при создании записи; если есть `id` — значит, есть и запись в БД.

После сохранения полученных данных созданный объект передаётся в объект form и становится доступен в шаблоне через атрибут `form.instance`. Получить `id` этого объекта можно так: `{{ form.instance.id }}`. 

Все остальные поля объекта тоже получим из form.instance — тег `{% with data=request.POST %}` поменяйте на `{% with data=form.instance %}`.

```html
...
{% with data=form.instance %}
  {% if form.is_valid %}
    ID записи в БД: {{ data.id }}
    <hr>
    <h2>Привет, {{ data.first_name }} {{ data.last_name }}</h2>
    {% if birthday_countdown == 0 %}
      <p>С днём рождения!</p>
    {% else %}
      <p>Осталось дней до дня рождения: {{ birthday_countdown }}!</p>
    {% endif %}
  {% endif %}
{% endwith %}
... 
```

Теперь после выполнения запроса на страницу будет выведен `id` созданного объекта:

![alt text](https://pictures.s3.yandex.net/resources/238_1682625501.png)


***
## Вывод объектов БД на веб-страницу

Чтобы увидеть все объекты, сохранённые в базе, — выведем их на веб-страницу *birthday/list/*; подготовим для неё шаблон, view-функцию и маршрут. 

Cоздайте в проекте шаблон *templates/birthday/birthday_list.html*. 

```html
<!-- templates/birthday/birthday_list.html -->
{% extends "base.html" %}

{% block content %}
  {% for birthday in birthdays %}
    <div> <!-- Обернём каждую запись в тег div -->
      <div> <!-- Текст обернём в отдельный вложенный div -->
        {{ birthday.first_name }} {{ birthday.last_name }} - {{ birthday.birthday }}
      </div>
      {% if not forloop.last %}
        <hr>
      {% endif %}
    </div>
  {% endfor %}
{% endblock %} 
```

Шаблон минималистичный: внутри блока `{% block content %}` объект birthdays перебирается в цикле, для каждого элемента `birthday` выводится имя, фамилия (если есть) и дата рождения; записи отделяются друг от друга горизонтальной линией `<hr>`. 